## Д/з

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [3]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

X_st = X.copy()
X_st[1, :] = calc_std_feat(X[1, :])
X_st[2, :] = calc_std_feat(X[2, :])
X_st[3, :] = calc_std_feat(X[3, :])

In [4]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [5]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    err = np.sum(err)
    return err

#### 1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [6]:
def calc_logloss(y, y_pred):
#   В ноль обращается выражение (1.0 - y_pred)
    y_pred[y_pred == 1] = 1-1e-12
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

#### 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [7]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [8]:
# Чем больше шаг я выбираю, тем меньше становится log loss, до бесконечности
W = eval_model(X_st, y, iterations=5000, alpha=0.5)

500 [ 0.51295149 -3.52799427 -2.30751655  6.62862132] 0.2396445385298005
1000 [ 0.99003968 -4.4739069  -3.50184385  9.11790023] 0.20436456104822748
1500 [ 1.36004524 -5.24499518 -4.41800057 11.08762583] 0.1824820722422445
2000 [ 1.66377414 -5.90329171 -5.17946051 12.74798172] 0.16699488670004922
2500 [ 1.92125494 -6.47778451 -5.83436059 14.18764762] 0.15538172117388738
3000 [ 2.14463478 -6.98733124 -6.41032319 15.46027175] 0.14632507336930287
3500 [ 2.34195752 -7.44521516 -6.92550556 16.60221895] 0.13904427863255084
4000 [ 2.5188181  -7.86115182 -7.39260609 17.63945533] 0.1330451723573654
4500 [ 2.67924745 -8.24243641 -7.82085282 18.59114884] 0.12800018223401377
5000 [ 2.82623283 -8.59466553 -8.21714641 19.4718142 ] 0.12368408748146356


#### 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [17]:
def calc_pred_proba (W, X):
    prob = sigmoid(np.dot(W, X))
    for i, pr in enumerate(prob):
        print(f'{i+1}: {pr:4f}')
    return prob
    
d = calc_pred_proba(W, X_st)

1: 0.377832
2: 0.020075
3: 1.000000
4: 0.000001
5: 0.905178
6: 0.072835
7: 1.000000
8: 0.032175
9: 0.586265
10: 0.999998


#### 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [10]:
def calc_pred(w, X):
    m = X.shape[1]
    
    y_predicted = np.zeros((1, m))
    y_predicted = np.squeeze(y_predicted)
    
    A = sigmoid(np.dot(w.T, X))
    
#     За порог отнесения к тому или иному классу примем вероятность 0.5
    for i in range(A.shape[0]):
        if (A[i] > 0.5): 
            y_predicted[i] = 1
        elif (A[i] <= 0.5):
            y_predicted[i] = 0
    
    return y_predicted

In [11]:
y_predicted = calc_pred(W,X_st)
y_predicted

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

#### 5*. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [12]:
# accuracy 
def accuracy(y_pred, y):
    return np.mean([y_pred == y])*100

accuracy(y_predicted, y)

100.0

In [13]:
# Матрица ошибок
def matr_err(y_pred, y):
    matr = np.zeros((2,2))
    TP = [num for i, num in enumerate(y_predicted) if num == y[i] and num == 1]
    matr[0][0] = len(TP)
    
    FP = [num for i, num in enumerate(y_predicted) if num != y[i] and num == 1]
    matr[0][1] = len(FP)
    
    FN = [num for i, num in enumerate(y_predicted) if num != y[i] and num == 0]
    matr[1][0] = len(FN)
    
    TN = [num for i, num in enumerate(y_predicted) if num == y[i] and num == 0]
    matr[1][1] = len(TN)
    matr = np.int64(matr)
    return matr

matr = matr_err(y_predicted, y)
print(matr)

[[5 0]
 [0 5]]


In [14]:
# Точность
def precision(matr_err):
    return matr_err[0][0] / (matr_err[0][0]+matr_err[0][1])

pr = precision(matr)
print(pr)

1.0


In [15]:
# Полнота
def recall (matr_err):
    return matr_err[0][0] / (matr_err[0][0]+matr_err[1][0])

rec = recall(matr)
print(rec)

1.0


In [16]:
# f1 score
def f1_score(precision, recall):
    return 2*precision*recall/(precision+recall)

f1_score(pr, rec)

1.0

#### 6. Могла ли модель переобучиться? Почему?

Мне кажется, что модель не может переобучиться. Понятие отступов: $M_{i}>0$, когда классификатор дает верный ответ, и $M_{i}<0$. Это говорит о том, что в целом линейная классификация уравновешивает себя, в итоге будет только уточняться.